## Intrinsic Dimension : Proof of Concept

This just demonstrates that the Intrinsic Dimension idea works for a simple MNIST-CNN model.

Of possible interest : This PyTorch version is 'generic' in the sense that it should be able
to take arbitrary parameterised models and investigate their Intrinsic Dimension (without
writing specialised `Module_IntrinsicDimension` classes for each module type).

In [6]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
import os

mnist_root = '/home/jevjev/Dropbox/Projects/datasets'

In [7]:
n_epochs = 5
batch_size, batch_log_interval, lr = 32, 600, 0.01
seed = 10

try_cuda = True

In [8]:
use_cuda = try_cuda and torch.cuda.is_available()
torch.manual_seed(seed)

device = torch.device("cuda" if use_cuda else "cpu")

In [9]:
loader_kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

mnist_transform = transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(mnist_root, train=True, download=True, transform=mnist_transform),
    batch_size=batch_size, shuffle=True, **loader_kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(mnist_root, train=False, transform=mnist_transform),
    batch_size=1000, shuffle=False, **loader_kwargs)

In [10]:
class RegularCNNModel(torch.nn.Module):
    def __init__(self):
        super(RegularCNNModel, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 16, kernel_size=3)
        self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=3)
        #self.conv2_drop = torch.nn.Dropout2d()
        self.fc1 = torch.nn.Linear(32*5*5, 50)
        self.fc2 = torch.nn.Linear(50, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d( x, 2)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = F.max_pool2d( x, 2)
        x = F.relu(x)
        
        #print(x.size())
        
        x = x.view(-1, 32*5*5)
        
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [11]:
def parameter_count(model):
    param_tot=0
    for name, param in model.named_parameters():
        if param.requires_grad:
            #print(name, param.data.size(), v_size)
            param_size = 1
            for d in list(param.data.size()):
                param_size *= d
            param_tot += param_size
    return param_tot    

In [12]:
def train(model, optimizer, epoch_num):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % batch_log_interval == 0:
            print('Train Epoch: {} [{: 6d}/{: 6d} ({:2.0f}%)]\tLoss: {:.4f}'.format(
                epoch_num, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model):
    model.eval()
    test_loss, correct = 0., 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    pct_correct = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), pct_correct))
    return pct_correct

In [13]:
def use_model(model):  # , **kwargs
    _model = model.to(device)
    #optimizer = optim.SGD(_model.parameters(), lr=args.lr, momentum=momentum)
    optimizer = torch.optim.Adam(_model.parameters(), lr=lr)
    return _model, optimizer

def get_stats_for(model, n_epochs=n_epochs):
    print("Total model parameters : %d" % (parameter_count(model),) )
    _model, optimizer = use_model(model)
    for epoch in range(1, n_epochs + 1):
        train(_model, optimizer, epoch)
        pct_correct = test(_model)
    return pct_correct
        
get_stats_for(RegularCNNModel())

Total model parameters : 45360
Train Epoch: 1 [     0/ 60000 ( 0%)]	Loss: 2.3573
Train Epoch: 1 [ 19200/ 60000 (32%)]	Loss: 0.3020
Train Epoch: 1 [ 38400/ 60000 (64%)]	Loss: 1.0018
Train Epoch: 1 [ 57600/ 60000 (96%)]	Loss: 0.4337


/home/jevjev/anaconda3/envs/dl/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1077, Accuracy: 9646/10000 (96.5%)

Train Epoch: 2 [     0/ 60000 ( 0%)]	Loss: 0.6631
Train Epoch: 2 [ 19200/ 60000 (32%)]	Loss: 0.4666
Train Epoch: 2 [ 38400/ 60000 (64%)]	Loss: 0.4551
Train Epoch: 2 [ 57600/ 60000 (96%)]	Loss: 0.4644

Test set: Average loss: 0.0995, Accuracy: 9699/10000 (97.0%)

Train Epoch: 3 [     0/ 60000 ( 0%)]	Loss: 0.2651
Train Epoch: 3 [ 19200/ 60000 (32%)]	Loss: 0.6946
Train Epoch: 3 [ 38400/ 60000 (64%)]	Loss: 0.4436
Train Epoch: 3 [ 57600/ 60000 (96%)]	Loss: 0.3653

Test set: Average loss: 0.1080, Accuracy: 9710/10000 (97.1%)

Train Epoch: 4 [     0/ 60000 ( 0%)]	Loss: 0.1098
Train Epoch: 4 [ 19200/ 60000 (32%)]	Loss: 0.3905
Train Epoch: 4 [ 38400/ 60000 (64%)]	Loss: 0.4489
Train Epoch: 4 [ 57600/ 60000 (96%)]	Loss: 0.6840

Test set: Average loss: 0.1075, Accuracy: 9701/10000 (97.0%)

Train Epoch: 5 [     0/ 60000 ( 0%)]	Loss: 0.2901


KeyboardInterrupt: 

In [14]:
class IntrinsicDimensionWrapper(torch.nn.Module):
    
    def __init__(self, module, intrinsic_dimension, verbose=False):
        """
        Wrapper to estimate the intrinsic dimensionality of the 
        objective landscape for a specific task given a specific model
        :param module: pytorch nn.Module
        :param intrinsic_dimension: dimensionality within which we search for solution
        :param verbose: if things should be printed out
        """
        super(IntrinsicDimensionWrapper, self).__init__()
        
        self.verbose = verbose
        
        self.m = [module] # Hide this from inspection by get_parameters()
        
        self.name_base_localname = []
        
        # Stores the initial value: \theta_{0}
        self.initial_value = dict()
        
        
        self.random_matrix = dict()
       
        V = torch.nn.Parameter( torch.zeros( (intrinsic_dimension,1) ).to(device) )
        self.register_parameter('V', V)
        v_size = (intrinsic_dimension, )
        
        # Iterates over parameters in the Neural Network
        for name, param in module.named_parameters():
            # If the parameter requires gradient update
            if param.requires_grad:
                
                if self.verbose:
                    print(name, param.data.size(), v_size)
                
                # Saves the initial values of the initialised parameters from param.data and sets them to no grad.
                # (initial values are the 'origin' of the search)
                self.initial_value[name] = v0 = torch.tensor(param.data, requires_grad=False).to(device)
                
                matrix_size = v0.size() + v_size
                self.random_matrix[name] = (
                    torch.randn(matrix_size, requires_grad=False).to(device)
                        / intrinsic_dimension**0.5
                )
                
                base, localname = module, name
                while '.' in localname:
                    prefix, localname = localname.split('.', 1)
                    #print(prefix, name)
                    base = base.__getattr__(prefix)
                self.name_base_localname.append( (name, base, localname) )

        for name,base,localname in self.name_base_localname:
            delattr(base, localname)

    def forward(self, x):   
        for name,base,localname in self.name_base_localname:
            #print(self.initial_value[name].size(), self.random_matrix[name].size(), self.V.size(), 
            #      torch.matmul(self.random_matrix[name], self.V).size())
            ray = torch.matmul(self.random_matrix[name], self.V)
            param = self.initial_value[name] + torch.squeeze(ray, -1)

            setattr(base, localname, param)

        module = self.m[0]
        x = module(x)
        return x

#model_single= torch.nn.Linear(3,4)
model_single = IntrinsicDimensionWrapper( torch.nn.Linear(3, 4), 10 )

#[p[0] for p in model_test.named_parameters()]
[ (p.view(-1), p.requires_grad) for p in model_single.parameters() ]
#model_test.initial_value

weight torch.Size([4, 3]) (10,)
bias torch.Size([4]) (10,)


/home/jevjev/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[(tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
         grad_fn=<ViewBackward>), True)]

In [15]:
x = torch.tensor( [1., 5., -1.25] ).to(device)

model_single(x)

tensor([0.5832, 1.1326, 0.2706, 4.2738], device='cuda:0',
       grad_fn=<AddBackward0>)

In [16]:
#model_test.V.data[3]=0.
model_single.V.data[3]+=.005
model_single(x)

tensor([0.5881, 1.1462, 0.2842, 4.2694], device='cuda:0',
       grad_fn=<AddBackward0>)

In [17]:
opt_test = torch.optim.Adam(model_single.parameters(), lr=lr)

model_single.train()

#data, target = data.to(device), target.to(device)
x_batch = torch.unsqueeze(x,0)
data, target = x_batch, torch.tensor( [1,] ).to(device)

opt_test.zero_grad()
output = model_single(data)
loss = F.nll_loss(output, target)
print(loss)
loss.backward()
opt_test.step()

model_single.V.requires_grad,
#model_single.m[0].weight.grad, model_single.m[0].weight.grad

tensor(-1.1462, device='cuda:0', grad_fn=<NllLossBackward>)


(True,)

In [18]:
## Now, let's build the CNN model with Intrinsic Dimension Wrapping...

intrinsic_dimension_guess = 5000

In [19]:
model_base = RegularCNNModel()
#[name for name,param in model_base.named_parameters()]

model_wrapped = IntrinsicDimensionWrapper( model_base, intrinsic_dimension_guess )
[name for name,param in model_wrapped.named_parameters()]
#[param for param in model_wrapped.parameters()]

conv1.weight torch.Size([16, 1, 3, 3]) (5000,)
conv1.bias torch.Size([16]) (5000,)
conv2.weight torch.Size([32, 16, 3, 3]) (5000,)
conv2.bias torch.Size([32]) (5000,)
fc1.weight torch.Size([50, 800]) (5000,)


/home/jevjev/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


fc1.bias torch.Size([50]) (5000,)
fc2.weight torch.Size([10, 50]) (5000,)
fc2.bias torch.Size([10]) (5000,)


['V']

get_stats_for( model_wrapped )

# Toy problem